**Cole Turner and Ethan Seal**

Fall 2019

CS343: Neural Networks

Project 3: Convolutional Neural Networks

In [16]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=7)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Global note: Make sure any debug printouts do not appear if `verbose=False`!**

## Task 4) Implement weight optimizers for gradient descent

To change the weights during training, we need an optimization algorithm to have our loss decrease over epochs as we learn the structure of the input patterns. Until now, we used **Stochastic gradient descent (SGD)**, which is the simplest algorithm. We will implement 3 popular algorithms:

- `SGD` (stochastic gradient descent)
- `SGD_Momentum` (stochastic gradient descent with momentum)
- `Adam` (Adaptive Moment Estimation)

Implement each of these according to the update equations (in `optimizer.py::update_weights` in each subclass). Let's use $w_t$ in the math below to represent the weights in a layer at time step $t$, $dw$ to represent the gradient of the weights in a layer, and $\eta$ represent the learning rate. We use vectorized notation below (update applies to all weights element-wise). Then:

**SGD**: 

$w_{t} = w_{t-1} - \eta \times dw$

**SGD (momentum)**:

$v_{t} = m \times v_{t-1} - \eta \times dw$

$w_{t} = w_{t-1} + v_t$

where $v_t$ is called the `velocity` at time $t$. At the first time step (0), velocity should be set to all zeros and have the same shape as $w$. $m$ is a constant that determines how much of the gradient obtained on the previous time step should factor into the weight update for the current time step.


**Adam**:

$m_{t} = \beta_1 \times m_{t-1} + (1 - \beta_1)\times dw$

$v_{t} = \beta_2 \times v_{t-1} + (1 - \beta_2)\times dw^2$

$n = m_{t} / \left (1-(\beta_1^t) \right )$

$u = v_{t} / \left (1-(\beta_2^t) \right )$

$w_{t} = w_{t-1} - \left ( \eta \times n \right ) / \left ( \sqrt(u) + \epsilon \right ) $


Like SGD (momentum), Adam records momentum terms $m$ and $v$. At time step 0, you should initialize them to zeros in an array equal in size to the weights. $n$ and $u$ are variables computed on each time step. The remaining quantities are constants. Note that $t$ keeps track of the integer time step, and needs to be incremented on each update. 

In [17]:
from optimizer import *

####  Test SGD

In [18]:
np.random.seed(0)

wts = np.arange(-3, 3, dtype=np.float64)
d_wts = np.random.randn(len(wts))

optimizer = SGD()
optimizer.prepare(wts, d_wts)

new_wts_1 = optimizer.update_weights()
new_wts_2 = optimizer.update_weights()

print(f'SGD: Wts after 1 iter {new_wts_1}')
print(f'SGD: Wts after 2 iter {new_wts_2}')

SGD: Wts after 1 iter [-3.1764052 -2.0400157 -1.0978738 -0.2240893  0.8132442  2.0977278]
SGD: Wts after 2 iter [-3.3528105 -2.0800314 -1.1957476 -0.4481786  0.6264884  2.1954556]


Output should be:

    SGD: Wts after 1 iter [-3.1764052 -2.0400157 -1.0978738 -0.2240893  0.8132442  2.0977278]
    SGD: Wts after 2 iter [-3.3528105 -2.0800314 -1.1957476 -0.4481786  0.6264884  2.1954556]

####  Test SGD_Momentum

In [19]:
np.random.seed(0)

wts = np.random.randn(3, 4)
d_wts = np.random.randn(3, 4)

optimizer = SGD_Momentum(lr=0.1, m=0.6)
optimizer.prepare(wts, d_wts)

new_wts_1 = optimizer.update_weights()
new_wts_2 = optimizer.update_weights()

print(f'SGD M: Wts after 1 iter\n{new_wts_1}')
print(f'SGD M: Wts after 2 iter\n{new_wts_2}')

SGD M: Wts after 1 iter
[[ 1.6879486  0.3879897  0.9343517  2.2075258]
 [ 1.7181501 -0.9567621  0.9187816 -0.0659476]
 [ 0.1520801  0.3452366  0.0576     1.52849  ]]
SGD M: Wts after 2 iter
[[ 1.5661825  0.3685217  0.8633335  2.1541379]
 [ 1.4790974 -0.9239367  0.8686908  0.0707077]
 [ 0.5605585  0.2406577 -0.0807098  1.6472364]]


Output should be:

    SGD M: Wts after 1 iter
    [[ 1.6879486  0.3879897  0.9343517  2.2075258]
     [ 1.7181501 -0.9567621  0.9187816 -0.0659476]
     [ 0.1520801  0.3452366  0.0576     1.52849  ]]
    SGD M: Wts after 2 iter
    [[ 1.5661825  0.3685217  0.8633335  2.1541379]
     [ 1.4790974 -0.9239367  0.8686908  0.0707077]
     [ 0.5605585  0.2406577 -0.0807098  1.6472364]]

####  Test Adam

In [20]:
np.random.seed(0)

wts = np.random.randn(3, 4)
d_wts = np.random.randn(3, 4)

optimizer = Adam(lr=0.1)
optimizer.prepare(wts, d_wts)

new_wts_1 = optimizer.update_weights()
new_wts_2 = optimizer.update_weights()
new_wts_3 = optimizer.update_weights()

print(f'Adam: Wts after 1 iter\n{new_wts_1}')
print(f'Adam: Wts after 2 iter\n{new_wts_2}')
print(f'Adam: Wts after 3 iter\n{new_wts_3}')

Adam: Wts after 1 iter
[[ 1.6640523  0.3001572  0.878738   2.1408932]
 [ 1.767558  -0.8772779  0.8500884 -0.0513572]
 [-0.0032189  0.3105985  0.0440436  1.5542735]]
Adam: Wts after 2 iter
[[ 1.5640523  0.2001572  0.778738   2.0408932]
 [ 1.667558  -0.7772779  0.7500884  0.0486428]
 [ 0.0967811  0.2105985 -0.0559564  1.6542735]]
Adam: Wts after 3 iter
[[ 1.4640523  0.1001572  0.678738   1.9408932]
 [ 1.567558  -0.6772779  0.6500884  0.1486428]
 [ 0.1967811  0.1105985 -0.1559564  1.7542735]]


Output should be:

    Adam: Wts after 1 iter
    [[ 1.6640523  0.3001572  0.878738   2.1408932]
     [ 1.767558  -0.8772779  0.8500884 -0.0513572]
     [-0.0032189  0.3105985  0.0440436  1.5542735]]
    Adam: Wts after 2 iter
    [[ 1.5640523  0.2001572  0.778738   2.0408932]
     [ 1.667558  -0.7772779  0.7500884  0.0486428]
     [ 0.0967811  0.2105985 -0.0559564  1.6542735]]
    Adam: Wts after 3 iter
    [[ 1.4640523  0.1001572  0.678738   1.9408932]
     [ 1.567558  -0.6772779  0.6500884  0.1486428]
     [ 0.1967811  0.1105985 -0.1559564  1.7542735]]  

## Task 5) Write network training methods

Implement methods in `network.py` to actually train the network, using all the building blocks that you have created. The methods to implement are:

- `predict`
- `fit`. Add an optional parameter `print_every=1` that controls the frequency (in iterations) with which to wait before printing out the loss and iteration number.

## Task 6) Overfitting a convolutional neural network

Usually we try to prevent overfitting, but we can use it as a valuable debugging tool to test out a complex backprop-style neural network. Assuming everything is working, it is almost always the case that we should be able to overfit a tiny dataset with a huge model with tons of parameters (i.e. your CNN). You will use this strategy to verify that your network is working.

Let's use a small amount of real data from STL-10. If everything is working properly, the network should overfit and you should see a significant drop in the loss from its starting value of ~2.3.

### 6a) Move your `preprocess_data.py` from the MLP project

Make the one following change:

- Re-arrange dimensions of `imgs` so that when it is returned, `shape=(Num imgs, RGB color chans, height, width)` (No longer flatten non-batch dimensions)

In [21]:
import load_stl10_dataset
import preprocess_data
from network import ConvNet4
import optimizer

### 6b) Load in STL-10 at 16x16 resolution

If you don't want to wait for STL-10 to download from the internet and resize, copy over your data and numpy folders from your MLP project.

**Notes:**
- You will need to download the new version of `load_stl10_dataset`.
- The different train/test split here won't work if you hard coded the proportions in your `create_splits` implementation! *This isn't catastrophic, it just means that it will take longer to compute accuracy on the validation set.*

In [22]:
# Download the STL-10 dataset from the internet, convert it to Numpy ndarray, resize to 16x16
# cache it locally on your computer for faster loading next time.
load_stl10_dataset.purge_cached_dataset()
stl_imgs, stl_labels = load_stl10_dataset.load(scale_fact=6)
# preprocess
stl_imgs, stl_labels = preprocess_data.preprocess_stl(stl_imgs, stl_labels)
# create splits
x_train, y_train, x_test, y_test, x_val, y_val, x_dev, y_dev = preprocess_data.create_splits(
    stl_imgs, stl_labels, n_train_samps=4548, n_test_samps=400, n_valid_samps=2, n_dev_samps=50)

print ('Train data shape: ', x_train.shape)
print ('Train labels shape: ', y_train.shape)
print ('Test data shape: ', x_test.shape)
print ('Test labels shape: ', y_test.shape)
print ('Validation data shape: ', x_val.shape)
print ('Validation labels shape: ', y_val.shape)
print ('dev data shape: ', x_dev.shape)
print ('dev labels shape: ', y_dev.shape)

classes = np.loadtxt(os.path.join('data', 'stl10_binary', 'class_names.txt'), dtype=str)

Images are: (5000, 96, 96, 3)
Labels are: (5000,)
Resizing 5000 images to 16x16...Done!
Saving Numpy arrays the images and labels to ./numpy...Done!
imgs.shape (5000, 16, 16, 3)
data.shape (5000, 768)
Train data shape:  (4548, 768)
Train labels shape:  (4548,)
Test data shape:  (400, 768)
Test labels shape:  (400,)
Validation data shape:  (2, 768)
Validation labels shape:  (2,)
dev data shape:  (50, 768)
dev labels shape:  (50,)


### 6c) Train and overfit the network on a small STL-10 sample with each optimizer

**Goal:** If your network works, you should see a drop in loss over epochs to 0.

In 3 seperate cells below

- Create 3 different `ConvNet4` networks.
- Compile each with a different optimizer (each net uses a different optimizer).
- Train each on the **dev** set and validate on the tiny validation set (we dont care about out-of-training-set performance here).

You will be making plots demonstrating the overfitting for each optimizer below. **You should train the nets with the same number of epochs such that at least 2/3 of them clearly show loss convergence to a small value; one optimizer may not converge yet, and that's ok**. Cut off the simulations based on the 2/3 that do converge.

Guidelines:

- Weight scales and learning rates of `1e-2` should work well.
- Start by testing the Adam optimizer.
- Remember that the input shape is (3, 16, 16). You need to specify this to the network constructor.
- The hyperparameters are up to you, though I wouldn't recommend a batch size that is too small (close to 1), otherwise it may be tricky to see whether the loss is actually decreasing on average.
- Decreasing `acc_freq` will make the `fit` function evaluate the training and validation accuracy more often. This is a computationally intensive process, so small values come with an increase in training time. On the other hand, checking the accuracy too infrequently means you won't know whether the network is trending toward overfitting the training data, which is what you're checking for.
- Each training session takes ~30 mins on my laptop.

**Caveat emptor:** Training convolutional networks is notoriously computationally intensive. If you experiment with hyperparameters, each training session may take several hours. Use the loss/accuracy print outs to quickly gauge whether your hyperparameter choices are getting your network to decrease in loss. Monitor print outs and interrupt the Jupyter kernel if things are not trending in the right direction. Consider using the Davis 102 iMacs if this is running too slow on your laptop.

In [33]:
wt_scale = 1e-2
lr = 1e-2
input_shape = (3, 16, 16)
mini_batch_sz = 10
n_epochs = 150

#preprocessing flattened it when we actually wanted it not flattened.
x_dev = x_dev.reshape(x_dev.shape[0], input_shape[0], input_shape[1], input_shape[2])
x_val = x_val.reshape(x_val.shape[0], input_shape[0], input_shape[1], input_shape[2])


In [34]:
# Adam

adam = ConvNet4(input_shape=input_shape, wt_scale=wt_scale, verbose=False)
adam.compile('adam')
adam.fit(x_dev, y_dev, x_val, y_val, mini_batch_sz=mini_batch_sz, n_epochs=n_epochs)

Starting to train...
750 iterations. 5 iter/epoch.
Iteration: 1/750.
Time taken for iteration 0: 1.665635108947754
Estimated time to complete: 1249.2263317108154
Iteration: 2/750.
Iteration: 3/750.
Iteration: 4/750.
Iteration: 5/750.
Iteration: 6/750.
Iteration: 7/750.
Iteration: 8/750.
Iteration: 9/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [2.2845796024885723, 2.2924845991086413, 2.2430353643670484]


-----------ACCURACIES-----------

  Train acc: 0.16, Val acc: 0.0


Iteration: 10/750.
Iteration: 11/750.
Iteration: 12/750.
Iteration: 13/750.
Iteration: 14/750.
Iteration: 15/750.
Iteration: 16/750.
Iteration: 17/750.
Iteration: 18/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [2.196307338513974, 2.1725589737331377, 1.8555820840431605]


-----------ACCURACIES-----------

  Train acc: 0.24, Val acc: 0.0


Iteration: 19/750.
Iteration: 20/750.
Iteration: 21/750.
Iteration: 22

Iteration: 183/750.
Iteration: 184/750.
Iteration: 185/750.
Iteration: 186/750.
Iteration: 187/750.
Iteration: 188/750.
Iteration: 189/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.2526039181014987, 0.2661376588799025, 0.29333143778269355]


-----------ACCURACIES-----------

  Train acc: 0.96, Val acc: 0.0


Iteration: 190/750.
Iteration: 191/750.
Iteration: 192/750.
Iteration: 193/750.
Iteration: 194/750.
Iteration: 195/750.
Iteration: 196/750.
Iteration: 197/750.
Iteration: 198/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.1112629225244871, 0.07248974144177063, 0.31392346579773295]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 199/750.
Iteration: 200/750.
Iteration: 201/750.
Iteration: 202/750.
Iteration: 203/750.
Iteration: 204/750.
Iteration: 205/750.
Iteration: 206/750.
Iteration: 207/750.

-------------LOSS HISTORIES-------------

L

Iteration: 363/750.
Iteration: 364/750.
Iteration: 365/750.
Iteration: 366/750.
Iteration: 367/750.
Iteration: 368/750.
Iteration: 369/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.016828954981043625, 0.008532854220705712, 0.014868278238738564]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 370/750.
Iteration: 371/750.
Iteration: 372/750.
Iteration: 373/750.
Iteration: 374/750.
Iteration: 375/750.
Iteration: 376/750.
Iteration: 377/750.
Iteration: 378/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.009769343830962701, 0.014373982427621002, 0.00897708285199268]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 379/750.
Iteration: 380/750.
Iteration: 381/750.
Iteration: 382/750.
Iteration: 383/750.
Iteration: 384/750.
Iteration: 385/750.
Iteration: 386/750.
Iteration: 387/750.

-------------LOSS HISTORIES---------

Iteration: 541/750.
Iteration: 542/750.
Iteration: 543/750.
Iteration: 544/750.
Iteration: 545/750.
Iteration: 546/750.
Iteration: 547/750.
Iteration: 548/750.
Iteration: 549/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.002019017040326986, 0.003892469532275133, 0.0035536293901333015]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 550/750.
Iteration: 551/750.
Iteration: 552/750.
Iteration: 553/750.
Iteration: 554/750.
Iteration: 555/750.
Iteration: 556/750.
Iteration: 557/750.
Iteration: 558/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.0033645742777648154, 0.0025017081009596756, 0.003921704589888853]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 559/750.
Iteration: 560/750.
Iteration: 561/750.
Iteration: 562/750.
Iteration: 563/750.
Iteration: 564/750.
Iteration: 565/750.
Iteration: 566/750.
Iteration: 56

  Train acc: 1.0, Val acc: 0.0


Iteration: 721/750.
Iteration: 722/750.
Iteration: 723/750.
Iteration: 724/750.
Iteration: 725/750.
Iteration: 726/750.
Iteration: 727/750.
Iteration: 728/750.
Iteration: 729/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.00160438701564368, 0.0015889317417497155, 0.0020780690656655564]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 730/750.
Iteration: 731/750.
Iteration: 732/750.
Iteration: 733/750.
Iteration: 734/750.
Iteration: 735/750.
Iteration: 736/750.
Iteration: 737/750.
Iteration: 738/750.

-------------LOSS HISTORIES-------------

Loss original: 2.3055575589923514
Loss latest three: [0.0010388349009595177, 0.0013960343425225312, 0.0021584754233142122]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 739/750.
Iteration: 740/750.
Iteration: 741/750.
Iteration: 742/750.
Iteration: 743/750.
Iteration: 744/750.
Iteration: 745/750.

In [35]:
# SGD-M
sgd_m = ConvNet4(input_shape=input_shape, wt_scale=wt_scale, verbose=False)
sgd_m.compile('sgd_momentum')
sgd_m.fit(x_dev, y_dev, x_val, y_val, mini_batch_sz=mini_batch_sz, n_epochs=n_epochs)

Starting to train...
750 iterations. 5 iter/epoch.
Iteration: 1/750.
Time taken for iteration 0: 1.6862642765045166
Estimated time to complete: 1264.6982073783875
Iteration: 2/750.
Iteration: 3/750.
Iteration: 4/750.
Iteration: 5/750.
Iteration: 6/750.
Iteration: 7/750.
Iteration: 8/750.
Iteration: 9/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.3083244218204695, 2.305725695413475, 2.297431716772699]


-----------ACCURACIES-----------

  Train acc: 0.16, Val acc: 0.0


Iteration: 10/750.
Iteration: 11/750.
Iteration: 12/750.
Iteration: 13/750.
Iteration: 14/750.
Iteration: 15/750.
Iteration: 16/750.
Iteration: 17/750.
Iteration: 18/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.3018450824131884, 2.3036695764710173, 2.3017066047698584]


-----------ACCURACIES-----------

  Train acc: 0.16, Val acc: 0.0


Iteration: 19/750.
Iteration: 20/750.
Iteration: 21/750.
Iteration: 22/7

Iteration: 185/750.
Iteration: 186/750.
Iteration: 187/750.
Iteration: 188/750.
Iteration: 189/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.2809092996558507, 2.2789688282647362, 2.28805322376837]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 190/750.
Iteration: 191/750.
Iteration: 192/750.
Iteration: 193/750.
Iteration: 194/750.
Iteration: 195/750.
Iteration: 196/750.
Iteration: 197/750.
Iteration: 198/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.2551803876097742, 2.268825191182065, 2.2279083463692317]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 199/750.
Iteration: 200/750.
Iteration: 201/750.
Iteration: 202/750.
Iteration: 203/750.
Iteration: 204/750.
Iteration: 205/750.
Iteration: 206/750.
Iteration: 207/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest thr

  Train acc: 0.22, Val acc: 0.0


Iteration: 370/750.
Iteration: 371/750.
Iteration: 372/750.
Iteration: 373/750.
Iteration: 374/750.
Iteration: 375/750.
Iteration: 376/750.
Iteration: 377/750.
Iteration: 378/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.24355648076423, 2.176654193386759, 2.343633754517829]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 379/750.
Iteration: 380/750.
Iteration: 381/750.
Iteration: 382/750.
Iteration: 383/750.
Iteration: 384/750.
Iteration: 385/750.
Iteration: 386/750.
Iteration: 387/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.258344127286521, 2.287439493420776, 2.2160262565649242]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 388/750.
Iteration: 389/750.
Iteration: 390/750.
Iteration: 391/750.
Iteration: 392/750.
Iteration: 393/750.
Iteration: 394/750.
Iteration: 395/750.


Iteration: 554/750.
Iteration: 555/750.
Iteration: 556/750.
Iteration: 557/750.
Iteration: 558/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.1614771451312933, 2.3313536288551155, 2.166155356616962]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 559/750.
Iteration: 560/750.
Iteration: 561/750.
Iteration: 562/750.
Iteration: 563/750.
Iteration: 564/750.
Iteration: 565/750.
Iteration: 566/750.
Iteration: 567/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.1527029769325354, 2.256560466136464, 2.298755900566209]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 568/750.
Iteration: 569/750.
Iteration: 570/750.
Iteration: 571/750.
Iteration: 572/750.
Iteration: 573/750.
Iteration: 574/750.
Iteration: 575/750.
Iteration: 576/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest thr

  Train acc: 0.22, Val acc: 0.0


Iteration: 739/750.
Iteration: 740/750.
Iteration: 741/750.
Iteration: 742/750.
Iteration: 743/750.
Iteration: 744/750.
Iteration: 745/750.
Iteration: 746/750.
Iteration: 747/750.

-------------LOSS HISTORIES-------------

Loss original: 2.309333752799659
Loss latest three: [2.3353460865512017, 2.3626829171958854, 2.1615144716245003]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 748/750.
Iteration: 749/750.
Iteration: 750/750.


----------------FINAL OUTPUT----------------
Loss history last threee: [2.0949775769564627, 2.1786421785573125, 2.2810184967981635]
  Train acc: 0.22, Val acc: 0.0


In [36]:
# SGD
sgd = ConvNet4(input_shape=input_shape, wt_scale=wt_scale, verbose=False)
sgd.compile('sgd')
sgd.fit(x_dev, y_dev, x_val, y_val, mini_batch_sz=mini_batch_sz, n_epochs=n_epochs)

Starting to train...
750 iterations. 5 iter/epoch.
Iteration: 1/750.
Time taken for iteration 0: 1.6721012592315674
Estimated time to complete: 1254.0759444236755
Iteration: 2/750.
Iteration: 3/750.
Iteration: 4/750.
Iteration: 5/750.
Iteration: 6/750.
Iteration: 7/750.
Iteration: 8/750.
Iteration: 9/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [2.2667797754626706, 2.275256568294465, 2.2556253335188514]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 10/750.
Iteration: 11/750.
Iteration: 12/750.
Iteration: 13/750.
Iteration: 14/750.
Iteration: 15/750.
Iteration: 16/750.
Iteration: 17/750.
Iteration: 18/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [2.2727310367417854, 2.204103899788421, 2.258343837284645]


-----------ACCURACIES-----------

  Train acc: 0.22, Val acc: 0.0


Iteration: 19/750.
Iteration: 20/750.
Iteration: 21/750.
Iteration: 22/75

Iteration: 185/750.
Iteration: 186/750.
Iteration: 187/750.
Iteration: 188/750.
Iteration: 189/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [1.7687937906780133, 1.5499046858392924, 1.7933401927001027]


-----------ACCURACIES-----------

  Train acc: 0.44, Val acc: 0.0


Iteration: 190/750.
Iteration: 191/750.
Iteration: 192/750.
Iteration: 193/750.
Iteration: 194/750.
Iteration: 195/750.
Iteration: 196/750.
Iteration: 197/750.
Iteration: 198/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [1.8762247744837355, 1.8434306913313045, 1.646268527799408]


-----------ACCURACIES-----------

  Train acc: 0.4, Val acc: 0.0


Iteration: 199/750.
Iteration: 200/750.
Iteration: 201/750.
Iteration: 202/750.
Iteration: 203/750.
Iteration: 204/750.
Iteration: 205/750.
Iteration: 206/750.
Iteration: 207/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest th

Iteration: 369/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.3295559392454376, 0.5024100581564189, 0.4312812816763762]


-----------ACCURACIES-----------

  Train acc: 0.92, Val acc: 0.0


Iteration: 370/750.
Iteration: 371/750.
Iteration: 372/750.
Iteration: 373/750.
Iteration: 374/750.
Iteration: 375/750.
Iteration: 376/750.
Iteration: 377/750.
Iteration: 378/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.17926499542018026, 0.08146063120636833, 0.14776578401052776]


-----------ACCURACIES-----------

  Train acc: 0.96, Val acc: 0.0


Iteration: 379/750.
Iteration: 380/750.
Iteration: 381/750.
Iteration: 382/750.
Iteration: 383/750.
Iteration: 384/750.
Iteration: 385/750.
Iteration: 386/750.
Iteration: 387/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.12690574396394314, 0.14626360697760457, 0.27095333847671027]


----

  Train acc: 1.0, Val acc: 0.0


Iteration: 550/750.
Iteration: 551/750.
Iteration: 552/750.
Iteration: 553/750.
Iteration: 554/750.
Iteration: 555/750.
Iteration: 556/750.
Iteration: 557/750.
Iteration: 558/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.023462035444168056, 0.0072876130475124055, 0.007217936771467708]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 559/750.
Iteration: 560/750.
Iteration: 561/750.
Iteration: 562/750.
Iteration: 563/750.
Iteration: 564/750.
Iteration: 565/750.
Iteration: 566/750.
Iteration: 567/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.011486809869905357, 0.014080469611488039, 0.006042325705134973]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 568/750.
Iteration: 569/750.
Iteration: 570/750.
Iteration: 571/750.
Iteration: 572/750.
Iteration: 573/750.
Iteration: 574/750.
Iter

  Train acc: 1.0, Val acc: 0.0


Iteration: 730/750.
Iteration: 731/750.
Iteration: 732/750.
Iteration: 733/750.
Iteration: 734/750.
Iteration: 735/750.
Iteration: 736/750.
Iteration: 737/750.
Iteration: 738/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.005925159649311415, 0.005571045170012228, 0.002954864165883129]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 739/750.
Iteration: 740/750.
Iteration: 741/750.
Iteration: 742/750.
Iteration: 743/750.
Iteration: 744/750.
Iteration: 745/750.
Iteration: 746/750.
Iteration: 747/750.

-------------LOSS HISTORIES-------------

Loss original: 2.304373733952622
Loss latest three: [0.00718462533908763, 0.004472624064811412, 0.005578008915382949]


-----------ACCURACIES-----------

  Train acc: 1.0, Val acc: 0.0


Iteration: 748/750.
Iteration: 749/750.
Iteration: 750/750.


----------------FINAL OUTPUT----------------
Loss history last threee: [0.0051878711

**Question 3**: Why does decreasing the mini-batch size make the loss print-outs more erratic?



### 6d) Evaluate the different optimizers

Make 2 "high quality" plots showing the following

- Plot the accuracy (y axis) for the three optimizers as a function of training epoch (x axis).
- Plot the loss (y axis) for the three optimizers as a function of training iteration (x axis).

A high quality plot consists of:
- A useful title
- X and Y axis labels
- A legend

**Question 4**: Which optimizer works best and why do think it is best?

**Question 5**: What is happening with the training set accuracy and why?

## Task 7: Training convolutional neural network on STL-10

### 7a) Load in STL-10 at 32x32 resolution

In [ ]:
# Download the STL-10 dataset from the internet, convert it to Numpy ndarray, resize to 32x32
# cache it locally on your computer for faster loading next time.
load_stl10_dataset.purge_cached_dataset()
stl_imgs, stl_labels = load_stl10_dataset.load(scale_fact=3)
# preprocess
stl_imgs, stl_labels = preprocess_data.preprocess_stl(stl_imgs, stl_labels)
# create splits
x_train, y_train, x_test, y_test, x_val, y_val, x_dev, y_dev = preprocess_data.create_splits(
    stl_imgs, stl_labels, n_train_samps=4548, n_test_samps=400, n_valid_samps=2, n_dev_samps=50)

print ('Train data shape: ', x_train.shape)
print ('Train labels shape: ', y_train.shape)
print ('Test data shape: ', x_test.shape)
print ('Test labels shape: ', y_test.shape)
print ('Validation data shape: ', x_val.shape)
print ('Validation labels shape: ', y_val.shape)
print ('dev data shape: ', x_dev.shape)
print ('dev labels shape: ', y_dev.shape)

classes = np.loadtxt(os.path.join('data', 'stl10_binary', 'class_names.txt'), dtype=str)

### 7b) Set up accelerated convolution and max pooling layers

As you may have noticed, we had to downsize STL-10 to 16x16 resolution to train the network on the dev set (N=50) in a reasonable amount of time. The training set is N=4000, how will we ever manage to process that amount of data!?

On one hand, this is an unfortunate inevitable reality of working with large ("big") datasets: you can easily find a dataset that is too time consuming to process for any computer, despite how fast/many CPU/GPUs it has.

On the other hand, we can do better for this project and STL-10 :) If you were to time (profile) different parts of the training process, you'd notice that largest bottleneck is convolution and max pooling operations (both forward/backward). You implemented those operations intuitively, which does not always yield the best performance. **By swapping out forward/backward convolution and maxpooling for implementations that use different algorithms (im2col, reshaping) that are compiled to C code, we will speed up training up by several orders of magnitude**.

Follow these steps to subsitute in the "accelerated" convolution and max pooling layers.

- Install the `cython` python package: `pip3 install cython` (or `pip3 install cython --user` if working in Davis 102)
- Dowload files `im2col_cython.pyx`, `accelerated_layer.py`, `setup.py` from the project website. Put them in your base project folder.
- Open terminal, `cd` to Project directory.
- Compile the im2col functions: `python3 setup.py build_ext --inplace`. A `.c` and `.so` file should have appeared in your project folder.
- Restart Jupyter Notebook kernel
- Create a class called `Conv4NetAccel` in `network.py` by copy-pasting the contents of `Conv4Net`. Import `accelerated_layer` at the top and replace the `Conv2D` and `MaxPool2D` layers with `Conv2DAccel` and `MaxPool2DAccel`.

### 7c) Training convolutional neural network on STL-10

You are now ready to train on the entire training set.

- Create a `Conv4NetAccel` object with hyperparameters of your choice.
- Your goal is to achieve 45% accuracy on the test and/or validation set.

Notes:

- I suggest using your intuition about hyperparameters and over/underfitting to guide your choice, rather than a grid search. This should not be overly challenging.
- Use the best / most efficient optimizer based on your prior analysis.
- It should take on the order of 1 sec per training iteration. If that's way off, seek help as something could be wrong with running the acclerated code.

In [ ]:
from network import ConvNet4Accel

### 7d) Analysis of STL-10 training quality

Use your trained network that achieves 45%+ accuracy on the test set to make "high quality" plots showing the following 

- Plot the accuracy of the training and validation sets as a function of training epoch. You may have to convert iterations to epochs.
- Plot the loss as a function of training iteration.

In [ ]:
plt.plot(netT.validation_acc_history)
plt.plot(netT.train_acc_history)
plt.show()

In [ ]:
plt.plot(netT.loss_history)
plt.show()

### 7f) Visualize layer weights

Run the following code and submit the inline image of the weight visualization of the 1st layer (convolutional layer) of the network.

**Note:**
- Setting optional parameter to `True` will let you save a .PNG file in your project folder of your weights. I'd suggest setting it to `False` unless look at your weights and they look like they are worth saving. You don't want a training run that produces undesirable weights to overwrite your good looking results!

In [ ]:
def plot_weights(wts, saveFig=True, filename='convWts_adam_overfit.png'):
    grid_sz = int(np.sqrt(len(wts)))
    plt.figure(figsize=(10,10))
    for x in range(grid_sz):
        for y in range(grid_sz):
            lin_ind = np.ravel_multi_index((x, y), dims=(grid_sz, grid_sz))
            plt.subplot(grid_sz, grid_sz, lin_ind+1)
            currImg = wts[lin_ind]
            low, high = np.min(currImg), np.max(currImg)
            currImg = 255*(currImg - low) / (high - low)
            currImg = currImg.astype('uint8')
            plt.imshow(currImg)
            plt.gca().axis('off')
    if saveFig:
        plt.savefig('convWts_adam_overfit.png')
    plt.show()

In [ ]:
# Subsitute your trained network below
# netT is my network's name
# You shouldn't see RGB noise
plot_weights(netT.layers[0].wts.transpose(0, 2, 3, 1), saveFig=False, filename='convWts_adam_train_20epoch.png')

**Question 6:** What do the learned filters look like? Does this make sense to you / is this what you expected? In which area of the brain do these filters resemble cell receptive fields?

Note: you should not see RGB "noise". If you do, and you pass the "overfit" test with the Adam optimizer, you probably need to increase the number of training epochs.

## Extensions

**General advice:** When making modifications for extensions, make small changes, then check to make sure you pass test code. Also, test out the network runtime on small examples before/after the changes. If you're not careful, the simulation time can become intractable really quickly!

**Remember:** One thorough extension usually is worth more than several "shallow" extensions.

### 0. Pedal to the metal: achieve high accuracy on STL-10

You can achieve higher (>50%) classification accuracy on the STL-10 test set. Find the hyperparameters to achieve this.

### 1. Experiment with different network architectures.

The design of the `Network` class is modular. As long as you're careful about shapes, adding/removing network layers (e.g. `Conv2D`, `Dense`, etc.) should be straight forward. Experiment with adding another sequence of `Conv2D` and `MaxPooling2D` layers. Add another `Dense` hidden layer before the output layer. How do the changes affect classification accuracy and loss? 

### 2. Experiment with different network hyperparameters.

Explore the affect one or more change below has on classification. Be careful about how the hyperparameters may affect the shape of network layers. Thorough analysis will get you more points (not try a few ad hoc values).

- Experiment with different numbers of hidden units in the Dense layers.
- Experiment different max pooling window sizes and strides.
- Experiment with kernel sizes (not 7x7). Can you get away with smaller ones? Do they perform just as well? What is the change in runtime like? What is the impact on their visualized appearance?
- Experiment with number of kernels in the convolutional layer. Is more/fewer better? What is the impact on their visualized appearance?

###  3. Add and test some training bells and whistles

Add features like early stopping, learning rate decay (learning rate at the end of an epoch becomes some fraction of its former value), etc and assess how they affect training loss convergence and accuracy. 

### 4. Additional optimizers

Research other optimizers used in backpropogation and implement one or more of them within the model structure. Compare its performance to ones you have implemented

### 5. Optimize your algorithms

Find the main performance bottlenecks in the network and improve your code to reduce runtime (e.g. reduce explicit for loops, increase vectorization, etc). Research faster algorithms to do operations like convolution and implement them. Given the complexity of the network, I suggest focusing on one area at a time and make sure everything you change passes the test code before proceeding. Quantify and discuss your performance improvements

### 6. Additional loss functions

Implement support for sigmoid, or another activation functions and associated losses. Test it out and compare with softmax/cross entropy. Make sure any necessary changes to the layer's gradient are made.

### 7. Additional datasets

Do classification and analyxe the results with an image dataset of your choice.

### 8. Performance analysis

Do a thorough comparative analysis of the non-accelerated network and accelerated networks with respect to runtime.